In [1]:
%cd ..

/home/elena/Documents/Project_jobs


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import re
from collections import Counter
import seaborn as sns
sns.set(style= "dark")
import project_jobs.helper_functions as hf

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

from sklearn.feature_selection import SelectKBest

from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score

In [3]:
data = hf.load_purified_data(r'data/NYC_Jobs.csv')
data.head()
# data.shape

,Job ID,Agency,Posting Type,Business Title,Civil Service Title,Title Code No,Level,Job Category,Full-Time/Part-Time indicator,Salary Range From,Salary Range To,Salary Frequency,Work Location,Division/Work Unit,Job Description,Minimum Qual Requirements,Preferred Skills,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date,Mean Salary
0,87990,department of business serv.,internal,account manager,contract reviewer office of l,40563,1,NaN,NaN,42405.00,65485.00,annual,110 william st. n y,strategy & analytics,division of economic & financial opportunity d...,1.a baccalaureate degree from an accredited co...,excellent interpersonal and organizational ski...,"salary range for this position is 42,405 - 45,...",NaN,NaN,NaN,NaN,new york city residency is generally required ...,2011-06-24,NaN,06242011,07022019,53945.0
1,97899,department of business serv.,internal,"executive director, business development",administrative business promot,10009,m3,NaN,f,60740.00,162014.00,annual,110 william st. n y,tech talent pipeline,the new york city department of small business...,1. a baccalaureate degree from an accredited c...,NaN,NaN,"in addition to applying through this website, ...",NaN,NaN,NaN,new york city residency is generally required ...,2012-01-26,NaN,01262012,07022019,111377.0
2,132292,nyc housing authority,external,maintenance worker - technical services-heatin...,maintenance worker,90698,0,maintenance & operations,f,51907.68,54580.32,annual,heating mgt-operations,management services department,"under direct supervision, assist in the routin...",1. three years of full-time satisfactory exper...,1. a high school diploma or ged. 2. cdl dri...,1. a motor vehicle drivers license valid in t...,click the apply now button.,NaN,NaN,NaN,nycha has no residency requirements.,2013-10-24,NaN,12122013,07022019,53244.0
3,132292,nyc housing authority,internal,maintenance worker - technical services-heatin...,maintenance worker,90698,0,maintenance & operations,f,51907.68,54580.32,annual,heating mgt-operations,management services department,"under direct supervision, assist in the routin...",1. three years of full-time satisfactory exper...,1. a high school diploma or ged. 2. cdl dri...,1. a motor vehicle drivers license valid in t...,click the apply now button.,NaN,NaN,NaN,nycha has no residency requirements.,2013-10-24,NaN,12122013,07022019,53244.0
4,133921,nyc housing authority,internal,temporary painter,painter,91830,0,maintenance & operations,f,35.00,35.00,hourly,dmp-contract & analysis unit,dept of management & planning,responsibilities of selected candidates will i...,1. five years of full-time satisfactory experi...,NaN,special note 1. this is a temporary assign...,click the apply now button.,NaN,NaN,NaN,nycha has no residency requirement.,2014-01-09,NaN,01082014,07022019,35.0


In [4]:
top_words = hf.top_words(data['Job Category'])
data.dropna(subset = ['Job Category'], axis='rows', inplace=True)
data = data.reset_index(drop=True)
print("data shape", '\t', data.shape)
df = pd.DataFrame(top_words, columns= ['top_words'])

y = []
for word in top_words:
    y.append(data[data['Job Category'].str.contains(word)]['Mean Salary'].mean())
y
df['mean_salary'] = pd.DataFrame(y)
df
df = df.sort_values('mean_salary', ascending=False)
df = df.reset_index(drop=True)
df

data shape 	 (3205, 28)


,top_words,mean_salary
0,innovation,87077.816040
1,data,87077.816040
2,technology,86976.466257
3,telecommunications,85528.145161
4,information,85528.145161
5,architecture,85434.542095
6,engineering,85434.542095
7,planning,85434.542095
8,communications,75426.314138
9,affairs,73993.821774


In [5]:
df = df.head(3)
df

,top_words,mean_salary
0,innovation,87077.816040
1,data,87077.816040
2,technology,86976.466257


In [6]:
list_IT = df['top_words'].tolist()
list_IT

['innovation', 'data', 'technology']

In [7]:
str_IT = '|'.join(list_IT)
print(str_IT)
top_job_cat = data[data['Job Category'].str.contains(str_IT)].copy()
top_job_cat.head()

innovation|data|technology


,Job ID,Agency,Posting Type,Business Title,Civil Service Title,Title Code No,Level,Job Category,Full-Time/Part-Time indicator,Salary Range From,Salary Range To,Salary Frequency,Work Location,Division/Work Unit,Job Description,Minimum Qual Requirements,Preferred Skills,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date,Mean Salary
11,167179,nyc employees retirement sys,external,"certified it administrator wan, level 4",cert. it administrator wan,13642,4,information technology & telecommunications,f,87203.0,131623.0,annual,"335 adams street, brooklyn ny",executive management,"oversees the design, installation, configurati...",1. a baccalaureate degree from an accredited c...,"minimum 5 years of experience planning, design...",NaN,to apply please click the apply now button. p...,NaN,NaN,NaN,new york city residency is not required for th...,2014-11-19,NaN,11192014,07022019,109413.0
12,167179,nyc employees retirement sys,internal,"certified it administrator wan, level 4",cert. it administrator wan,13642,4,information technology & telecommunications,f,87203.0,131623.0,annual,"335 adams street, brooklyn ny",executive management,"oversees the design, installation, configurati...",1. a baccalaureate degree from an accredited c...,"minimum 5 years of experience planning, design...",NaN,to apply please click the apply now button. p...,NaN,NaN,NaN,new york city residency is not required for th...,2014-11-19,NaN,11192014,07022019,109413.0
16,171944,dept of info tech & telecomm,internal,"deputy director, audits and accounts",computer assoc tech supp,13611,1,"finance, accounting, & procurement information...",NaN,43292.0,53000.0,annual,75 park place new york ny,financial services,"doitt provides for the sustained, efficient an...",qualification requirements 1. a baccalaureate...,the successful candidate should possess the fo...,NaN,for doitt employees only please go to employe...,day,"ny, ny",NaN,new york city residency is generally required ...,2014-10-30,NaN,10302014,07022019,48146.0
19,177048,dept of info tech & telecomm,internal,application support reporting specialist,computer specialist software,13632,3,information technology & telecommunications,NaN,81290.0,95896.0,annual,2 metro tech,develop & integration services,"doitt provides for the sustained, efficient an...",1 a baccalaureate degree from an accredited co...,the preferred candidate should possess the fol...,NaN,for doitt employees only please go to emplo...,day - due to the necessary technical support d...,"brooklyn, ny",NaN,new york city residency is not required for th...,2014-12-02,NaN,12022014,07022019,88593.0
22,183112,dept of info tech & telecomm,internal,gis analyst,computer programmer analyst,13651,1,information technology & telecommunications,f,45174.0,62370.0,annual,2 metro tech,develop & integration services,"doitt provides for the sustained, efficient an...",1 a baccalaureate degree from an accredited co...,the successful candidate should possess the fo...,NaN,for doitt employees only please go to employe...,due to the necessary technical support duties ...,"brooklyn, ny",NaN,new york city residency is not required for th...,2015-02-12,NaN,02122015,07022019,53772.0


In [8]:
list_top_civ = hf.top_words(data['Civil Service Title'])[:10]
str_top_civ = '|'.join(list_top_civ)
print(str_top_civ)
filter1 = top_job_cat[top_job_cat['Civil Service Title'].str.contains(str_top_civ)].copy()
print(filter1.shape)
filter1.head()
# filter1['Business Title'].unique()[:10]
# hf.top_words(filter1['Business Title'])

administrative|manager|engineer|associate|community|project|analyst|coordinator|specialist|city
(320, 28)


,Job ID,Agency,Posting Type,Business Title,Civil Service Title,Title Code No,Level,Job Category,Full-Time/Part-Time indicator,Salary Range From,Salary Range To,Salary Frequency,Work Location,Division/Work Unit,Job Description,Minimum Qual Requirements,Preferred Skills,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date,Mean Salary
19,177048,dept of info tech & telecomm,internal,application support reporting specialist,computer specialist software,13632,3,information technology & telecommunications,NaN,81290.0,95896.0,annual,2 metro tech,develop & integration services,"doitt provides for the sustained, efficient an...",1 a baccalaureate degree from an accredited co...,the preferred candidate should possess the fol...,NaN,for doitt employees only please go to emplo...,day - due to the necessary technical support d...,"brooklyn, ny",NaN,new york city residency is not required for th...,2014-12-02,NaN,12022014,07022019,88593.0
22,183112,dept of info tech & telecomm,internal,gis analyst,computer programmer analyst,13651,1,information technology & telecommunications,f,45174.0,62370.0,annual,2 metro tech,develop & integration services,"doitt provides for the sustained, efficient an...",1 a baccalaureate degree from an accredited co...,the successful candidate should possess the fo...,NaN,for doitt employees only please go to employe...,due to the necessary technical support duties ...,"brooklyn, ny",NaN,new york city residency is not required for th...,2015-02-12,NaN,02122015,07022019,53772.0
24,184328,dept of info tech & telecomm,internal,"director, infrastructure build coordinator",computer systems manager,10050,m1,information technology & telecommunications,NaN,49492.0,100000.0,annual,2 metro tech,public safety it services,"doitt provides for the sustained, efficient an...",1. a masters degree in computer science from a...,the successful candidate should possess the fo...,NaN,for doitt employees only please go to employe...,day - due to the necessary technical managemen...,"brooklyn, ny",NaN,new york city residency is not required for th...,2015-02-19,NaN,02192015,07022019,74746.0
57,229837,dept of info tech & telecomm,internal,senior quality oversight analyst,computer systems manager,10050,m2,information technology & telecommunications,f,58675.0,125000.0,annual,75 park place new york ny,general counsel,"doitt provides for the sustained, efficient an...",1. a masters degree in computer science from a...,the successful candidate possess the following...,NaN,for doitt employees only please go to employe...,day,manhattan,NaN,new york city residency is not required for th...,2016-02-17,NaN,02172016,07022019,91837.5
84,239204,admin for childrens svcs,external,child welfare analyst,city research scientist,21744,2,information technology & telecommunications po...,f,68239.0,85644.0,annual,"150 william street, new york n",srm eligrptsys spprt-p&p,the child welfare city research scientist will...,"1. for assignment level i only physical, biol...",critical to this position is strong technical ...,section 424-a of the new york social services ...,click on the apply now button.,NaN,NaN,NaN,new york city residency is generally required ...,2016-05-17,NaN,05172016,07022019,76941.5


In [9]:
filter1.loc[:, 'is_it'] = 1
filter1.head()
not_it = data[~data['Job ID'].isin(filter1['Job ID'].values.tolist())].copy()
not_it.shape
not_it.loc[:, 'is_it'] = 0
filtered_full = pd.concat([filter1, not_it], ignore_index=True)
print(filtered_full.shape)
# filtered_full['is_it'].value_counts()
filtered_full.head()

(3205, 29)


,Job ID,Agency,Posting Type,Business Title,Civil Service Title,Title Code No,Level,Job Category,Full-Time/Part-Time indicator,Salary Range From,Salary Range To,Salary Frequency,Work Location,Division/Work Unit,Job Description,Minimum Qual Requirements,Preferred Skills,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date,Mean Salary,is_it
0,177048,dept of info tech & telecomm,internal,application support reporting specialist,computer specialist software,13632,3,information technology & telecommunications,NaN,81290.0,95896.0,annual,2 metro tech,develop & integration services,"doitt provides for the sustained, efficient an...",1 a baccalaureate degree from an accredited co...,the preferred candidate should possess the fol...,NaN,for doitt employees only please go to emplo...,day - due to the necessary technical support d...,"brooklyn, ny",NaN,new york city residency is not required for th...,2014-12-02,NaN,12022014,07022019,88593.0,1
1,183112,dept of info tech & telecomm,internal,gis analyst,computer programmer analyst,13651,1,information technology & telecommunications,f,45174.0,62370.0,annual,2 metro tech,develop & integration services,"doitt provides for the sustained, efficient an...",1 a baccalaureate degree from an accredited co...,the successful candidate should possess the fo...,NaN,for doitt employees only please go to employe...,due to the necessary technical support duties ...,"brooklyn, ny",NaN,new york city residency is not required for th...,2015-02-12,NaN,02122015,07022019,53772.0,1
2,184328,dept of info tech & telecomm,internal,"director, infrastructure build coordinator",computer systems manager,10050,m1,information technology & telecommunications,NaN,49492.0,100000.0,annual,2 metro tech,public safety it services,"doitt provides for the sustained, efficient an...",1. a masters degree in computer science from a...,the successful candidate should possess the fo...,NaN,for doitt employees only please go to employe...,day - due to the necessary technical managemen...,"brooklyn, ny",NaN,new york city residency is not required for th...,2015-02-19,NaN,02192015,07022019,74746.0,1
3,229837,dept of info tech & telecomm,internal,senior quality oversight analyst,computer systems manager,10050,m2,information technology & telecommunications,f,58675.0,125000.0,annual,75 park place new york ny,general counsel,"doitt provides for the sustained, efficient an...",1. a masters degree in computer science from a...,the successful candidate possess the following...,NaN,for doitt employees only please go to employe...,day,manhattan,NaN,new york city residency is not required for th...,2016-02-17,NaN,02172016,07022019,91837.5,1
4,239204,admin for childrens svcs,external,child welfare analyst,city research scientist,21744,2,information technology & telecommunications po...,f,68239.0,85644.0,annual,"150 william street, new york n",srm eligrptsys spprt-p&p,the child welfare city research scientist will...,"1. for assignment level i only physical, biol...",critical to this position is strong technical ...,section 424-a of the new york social services ...,click on the apply now button.,NaN,NaN,NaN,new york city residency is generally required ...,2016-05-17,NaN,05172016,07022019,76941.5,1


In [10]:
model1 = filtered_full[['Job ID', 'is_it', 'Business Title']].copy()
model1.head()

,Job ID,is_it,Business Title
0,177048,1,application support reporting specialist
1,183112,1,gis analyst
2,184328,1,"director, infrastructure build coordinator"
3,229837,1,senior quality oversight analyst
4,239204,1,child welfare analyst


In [11]:
vectorizer = CountVectorizer(binary=True)
# bow = bag of words
bow = vectorizer.fit_transform(model1['Business Title']) #разреженная матрица
print(bow.shape)
print(type(bow))
df_bow = pd.DataFrame(bow.todense(), columns = vectorizer.get_feature_names())
# df_bow
# print(vectorizer.get_feature_names())
pd.concat([model1, df_bow], axis=1, sort=False).head()
# df_bow[(df_bow['application'] == 1) & (df_bow['reporting'] == 1)]

(3205, 984)
<class 'scipy.sparse.csr.csr_matrix'>


,Job ID,is_it,Business Title,02,03,1515,2020,311,40,421,abatement,ac1,academy,accelerator,acceptance,access,account,accountability,accountable,accountant,accounting,accounts,ace,action,ada,admin,adminiistrative,administration,administrative,administrator,administrators,adoption,adviser,advisor,advisory,advocate,affairs,affordability,agencies,agency,agent,aging,agreements,aide,aidesecurity,air,alcohol,alternative,alternatives,amendments,analyses,analysis,analyst,analystcoordinator,analysts,analytics,and,android,annual,aot,apa,appeals,application,applications,appraiser,apprentice,approval,architect,architectdeveloper,architectsenior,area,areas,art,arterial,arts,asbestos,assessment,assessments,asset,assets,assistance,assistant,assoc,associate,associateclerical,asst,assurance,attorney,audit,auditor,audits,auth,authority,auto,automation,automotive,bank,based,bedc,being,benefits,best,bid,billing,bim,bimcad,blade,bluebelt,board,bookkeeper,borough,branch,bridge,bronx,brooklyn,bts,budget,budgeting,build,building,buprenorphine,bureau,burner,business,buyer,cad,cadd,call,campaign,candidacy,candidate,capacity,capital,care,caretaker,cartography,case,catch,catholic,ccc,census,center,centered,central,cert,certification,certified,chaplain,charge,chemist,chief,child,childcare,childhood,children,childrens,chronic,citizenship,citrix,city,citytime,citywide,civil,civilian,civilianization,claim,claims,classification,clerical,clerk,client,climber,clinic,clinical,cloud,co,coach,coc,cod,code,collections,college,command,commander,commercial,commission,commissioner,commissionermental,commissioners,communicable,communications,community,compliance,computer,conference,confidential,conflict,connections,consortium,constituency,construction,consultant,consultation,content,continued,contract,contracting,contracts,control,controls,cook,coordination,coordinator,coordinators,corp,corporate,corporation,correction,correctional,cosntruction,cost,cota,counsel,counselor,counter,court,cpm,cr,cranes,credit,crew,crime,criminalist,crm,crmportal,cso,custodial,custodian,customer,customized,cyber,cybersecurity,dacco,dafn,dam,damage,data,database,dcas,ddpc,debt,deduction,delivery,department,deputy,derricks,design,designated,designer,desk,desktop,desktopfield,determination,dev,developer,developerprogrammer,development,device,devleloperanalyst,dfta,diem,diesel,digital,direct,director,directory,disability,disciplinary,disclosure,disease,diseases,distribution,district,diversity,division,dockbuilder,doctor,doe,doitt,door,downstate,drainage,drie,driver,drug,duplication,dycd,dynamic,early,ecological,economic,editor,editorial,education,educator,edx,eeo,efficiency,eh,ehs,ehso,electric,electrical,electrician,elevator,eligibility,ella,email,emergency,emissions,employee,employer,employmentlabor,energy,enforcement,engagement,engineer,engineeraccountable,engineering,engineers,enrollment,enterprise,entry,enviromental,environmental,eot,epidem,epidemiology,epmis,equipment,equity,escalation,eshare,estate,estimating,estimator,evaluation,evaluator,evp,examination,examiner,examining,exchangewindows,executive,exeutive,expense,expert,exterminator,external,extremely,facilitator,facilities,facility,failla,faith,families,family,fapjji,far,farm,fellow,fellowship,felony,fiduciary,field,file,finance,financial,financing,finishing,fire,first,fisa,fiscal,fitness,fixed,fleet,fms,for,force,forecasting,forensic,forester,forestry,foster,framework,franchise,freeze,front,fsot,ft,full,furniture,gardener,gbhu,general,generalbackground,generalist,generation,geotechnical,gis,governance,government,graduate,grant,grants,graphic,green,groundskeeper,group,guard,ha,hardware,harlem,hazard,hazards,head,health,healthcare,healthsh,healthy,hearing,hearings,heat,heating,help,helpdesk,helper,hepatitis,high,higher,highway,hiring,hiv,hivaids,hld,home,homes,horticulture,hot,house,housing,hqe,hr,hris,hrs,hsu,hts,hud,human,humanitarian,hygiene,hygienist,ia,iaqot,ib,icsu,identification,identity,ii,iii,implementation,improvement,in,inci

In [12]:
params = {'C': np.logspace(-5, 5, 11)}
# С = 1/lambda, lambda - regularization param

logreg = LogisticRegression(solver='lbfgs')
cv = GridSearchCV(logreg, params, n_jobs=-1, scoring='roc_auc', cv=5, return_train_score=True)
cv.fit(bow, model1.is_it);

In [13]:
pd.DataFrame(cv.cv_results_)[['mean_test_score', 'params']].sort_values('mean_test_score', ascending=False)

,mean_test_score,params
5,0.862625,{'C': 1.0}
6,0.851024,{'C': 10.0}
4,0.841432,{'C': 0.1}
7,0.823050,{'C': 100.0}
8,0.814677,{'C': 1000.0}
9,0.810772,{'C': 10000.0}
10,0.808395,{'C': 100000.0}
3,0.807268,{'C': 0.01}
2,0.795906,{'C': 0.001}
1,0.795218,{'C': 0.0001}


In [14]:
logreg = LogisticRegression(solver='lbfgs')
np.mean(cross_val_score(logreg, bow, model1.is_it, scoring='roc_auc', cv=5))

0.8626245667244368

In [15]:
cv.best_estimator_

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [16]:
def get_top_it(logreg, n):
    indices = np.zeros(bow.shape[1]) # создаем массив длинной по кол-ву столбцов в bow, заполняем нулями
    for i in np.argsort(logreg.coef_)[:, -n:]: # для n последних индексов из осортированного списка
#         выдает список индексов, которые отсортируют logreg.coef_
#         logreg.coef_ - массив весов для каждого столбца.
        indices[i] = 1 #заменить значение в массиве под этими индексами на 1
    
    return vectorizer.inverse_transform(indices)[0][::-1] #возвращает список названий столбцов, 
#отвечающих этим индексам в обратном порядке т.к. [::-1] 

def get_top_nonit(logreg, n):
    indices = np.zeros(bow.shape[1])
    for i in np.argsort(logreg.coef_)[:, :n]:
        indices[i] = 1
    
    return vectorizer.inverse_transform(indices)[0][::-1]

In [17]:
top = pd.DataFrame([get_top_it(cv.best_estimator_, 6),
                    get_top_nonit(cv.best_estimator_, 6)]).T

top.columns = ['IT', 'noIT']
top

,IT,noIT
0,technical,program
1,security,policy
2,reporting,investment
3,network,certified
4,developer,bureau
5,computer,administration


In [18]:
k_best = SelectKBest(k=100) # кроме кол-ва указывается еще метрика, по дефолту f_classif
bow_k_best = k_best.fit_transform(bow, model1.is_it)
print(bow[:5])
print(bow_k_best[:5])
# print(bow_k_best)
bow_k_best.shape #возвращает матрицу со всеми строками и 100 лучшими фичами

  (0, 59)	1
  (0, 787)	1
  (0, 853)	1
  (0, 884)	1
  (1, 49)	1
  (1, 407)	1
  (2, 115)	1
  (2, 209)	1
  (2, 271)	1
  (2, 482)	1
  (3, 49)	1
  (3, 651)	1
  (3, 757)	1
  (3, 830)	1
  (4, 49)	1
  (4, 148)	1
  (4, 968)	1
  (0, 88)	1
  (0, 85)	1
  (0, 75)	1
  (0, 1)	1
  (1, 0)	1
  (3, 81)	1
  (3, 0)	1
  (4, 0)	1


(3205, 100)

In [19]:
cols = k_best.get_support(indices=True)
print(cols)
columns = []
for i in cols:
    columns.append(vectorizer.get_feature_names()[i])
print(columns)

[ 49  59  60  66  78  84 101 112 118 120 128 141 155 180 190 200 208 221
 231 237 238 243 244 251 258 261 262 267 302 314 367 379 398 419 424 432
 465 471 472 480 484 494 496 511 515 530 536 540 556 559 560 568 572 573
 580 598 600 601 610 616 617 633 676 677 706 708 727 739 740 748 752 756
 769 770 783 787 798 813 815 819 829 830 840 845 846 853 858 882 884 892
 894 900 908 913 917 944 952 960 967 978]
['analyst', 'application', 'applications', 'architectsenior', 'assistant', 'attorney', 'bim', 'bts', 'bureau', 'business', 'capacity', 'cert', 'citrix', 'commander', 'computer', 'content', 'coordination', 'counter', 'crmportal', 'cyber', 'cybersecurity', 'data', 'database', 'deputy', 'desktopfield', 'developer', 'developerprogrammer', 'diem', 'edx', 'email', 'fellow', 'fisa', 'full', 'hardware', 'health', 'help', 'identity', 'incident', 'incidentproblemchange', 'information', 'initiatives', 'integration', 'intelligence', 'ios', 'it', 'lamp', 'lead', 'learning', 'linux', 'logging', 'logi

In [59]:
df_bow_k_best = pd.DataFrame(bow_k_best.todense(), columns = columns)
df_bow_k_best.head()
# for i in df_bow_k_best.shape[0]:
#     df_bow_k_best.loc[0]
for col in df_bow_k_best.iloc[0].to_frame().T.columns():
    df_bow_k_best
# df_bow_k_best.shape[0]

,analyst,application,applications,architectsenior,assistant,attorney,bim,bts,bureau,business,capacity,cert,citrix,commander,computer,content,coordination,counter,crmportal,cyber,cybersecurity,data,database,deputy,desktopfield,developer,developerprogrammer,diem,edx,email,fellow,fisa,full,hardware,health,help,identity,incident,incidentproblemchange,information,initiatives,integration,intelligence,ios,it,lamp,lead,learning,linux,logging,logic,mainframe,management,manager,master,middleware,mobile,modeling,mwbe,net,network,of,peoplesoft,per,portal,positions,process,program,programmer,protection,psac,qa,record,recording,reliability,reporting,resiliency,saas,salesforce,scheduler,security,senior,sharepoint,software,solutions,specialist,stack,supplies,support,systems,task,technical,tester,threat,tivoli,upstate,vendor,vulnerability,web,workload
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [21]:
logreg = LogisticRegression(solver='lbfgs')
np.mean(cross_val_score(logreg, bow_k_best, model1.is_it, scoring='roc_auc', cv=5))
# если второй раз делать векторайзер с ngram_range=(1, 4), то качество становится хуже чем на грид серч кросс валидации
# если с предыдущими настройками, то лучше.

0.870499350086655

In [22]:
x_train, x_test, y_train, y_test = train_test_split(bow, model1.is_it)
print(x_train.shape)
print(x_test.shape)

(2403, 984)
(802, 984)


In [23]:
k_best = SelectKBest(k=100)
x_train_k_best = k_best.fit_transform(x_train, y_train)
x_test_k_best = k_best.transform(x_test)

/home/elena/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] are constant.
  UserWarning)
/home/elena/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [24]:
logreg = LogisticRegression(solver='lbfgs')
logreg.fit(x_train_k_best, y_train)
roc_auc_score(y_test, logreg.predict_proba(x_test_k_best)[:, 1])

0.8937193261705796

In [25]:
predictions = logreg.predict(bow_k_best)
len(predictions)
# x_train_k_best
df_k_best = pd.DataFrame(bow_k_best.todense(), columns=columns)
df_k_best['y'] = model1.is_it.copy()
df_k_best['y_pred'] = predictions
df_k_best.head()
df_k_best['true_pred'] = (df_k_best.y == df_k_best.y_pred).astype(int)
# df_k_best.drop(columns = ['y', 'y_pred'], inplace = True)
df_k_best.head()
df_k_best.shape
df_k_best = pd.concat([df_k_best, model1], axis=1, sort=False)
df_k_best.head()

,analyst,application,applications,architectsenior,assistant,attorney,bim,bts,bureau,business,capacity,cert,citrix,commander,computer,content,coordination,counter,crmportal,cyber,cybersecurity,data,database,deputy,desktopfield,developer,developerprogrammer,diem,edx,email,fellow,fisa,full,hardware,health,help,identity,incident,incidentproblemchange,information,initiatives,integration,intelligence,ios,it,lamp,lead,learning,linux,logging,logic,mainframe,management,manager,master,middleware,mobile,modeling,mwbe,net,network,of,peoplesoft,per,portal,positions,process,program,programmer,protection,psac,qa,record,recording,reliability,reporting,resiliency,saas,salesforce,scheduler,security,senior,sharepoint,software,solutions,specialist,stack,supplies,support,systems,task,technical,tester,threat,tivoli,upstate,vendor,vulnerability,web,workload,y,y_pred,true_pred,Job ID,is_it,Business Title
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,177048,1,application support reporting specialist
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,183112,1,gis analyst
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,184328,1,"director, infrastructure build coordinator"
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,229837,1,senior quality oversight analyst
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,239204,1,child welfare analyst


In [35]:
df_k_best['prob_pos'] = logreg.predict_proba(bow_k_best)[:,1]
df_k_best['prob_neg'] = logreg.predict_proba(bow_k_best)[:,0]
df_k_best.head()


,analyst,application,applications,architectsenior,assistant,attorney,bim,bts,bureau,business,capacity,cert,citrix,commander,computer,content,coordination,counter,crmportal,cyber,cybersecurity,data,database,deputy,desktopfield,developer,developerprogrammer,diem,edx,email,fellow,fisa,full,hardware,health,help,identity,incident,incidentproblemchange,information,initiatives,integration,intelligence,ios,it,lamp,lead,learning,linux,logging,logic,mainframe,management,manager,master,middleware,mobile,modeling,mwbe,net,network,of,peoplesoft,per,portal,positions,process,program,programmer,protection,psac,qa,record,recording,reliability,reporting,resiliency,saas,salesforce,scheduler,security,senior,sharepoint,software,solutions,specialist,stack,supplies,support,systems,task,technical,tester,threat,tivoli,upstate,vendor,vulnerability,web,workload,y,y_pred,true_pred,Job ID,is_it,Business Title,prob_pos,prob_neg
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,177048,1,application support reporting specialist,0.591830,0.408170
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,183112,1,gis analyst,0.069033,0.930967
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,184328,1,"director, infrastructure build coordinator",0.035914,0.964086
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,229837,1,senior quality oversight analyst,0.144724,0.855276
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,239204,1,child welfare analyst,0.069033,0.930967


In [27]:
true_pred = df_k_best[df_k_best['true_pred'] == 1]
false_pred = df_k_best[df_k_best['true_pred'] == 0]

In [28]:
# уверенно относит к классу 1
true_positive = true_pred[true_pred['y_pred'] == 1].sort_values(by = 'prob_pos', ascending=False).head(3)
print(true_positive['Business Title'].tolist())
true_positive

['.net developer - financial systems', '.net developer - financial systems', 'e-learning content developer per diem']


,analyst,application,applications,architectsenior,assistant,attorney,bim,bts,bureau,business,capacity,cert,citrix,commander,computer,content,coordination,counter,crmportal,cyber,cybersecurity,data,database,deputy,desktopfield,developer,developerprogrammer,diem,edx,email,fellow,fisa,full,hardware,health,help,identity,incident,incidentproblemchange,information,initiatives,integration,intelligence,ios,it,lamp,lead,learning,linux,logging,logic,mainframe,management,manager,master,middleware,mobile,modeling,mwbe,net,network,of,peoplesoft,per,portal,positions,process,program,programmer,protection,psac,qa,record,recording,reliability,reporting,resiliency,saas,salesforce,scheduler,security,senior,sharepoint,software,solutions,specialist,stack,supplies,support,systems,task,technical,tester,threat,tivoli,upstate,vendor,vulnerability,web,workload,y,y_pred,true_pred,Job ID,is_it,Business Title,prob_pos,prob_neg
217,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,387746,1,.net developer - financial systems,0.957905,0.042095
216,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,387746,1,.net developer - financial systems,0.957905,0.042095
33,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,307249,1,e-learning content developer per diem,0.932512,0.067488


In [29]:
# уверенно относит к классу 0
true_negative = true_pred[true_pred['y_pred'] == 0].sort_values(by = 'prob_neg', ascending=False).head(3)
print(true_negative['Business Title'].tolist())
true_negative

['program executive', 'program coordinator', 'rentalfleet program processor']


,analyst,application,applications,architectsenior,assistant,attorney,bim,bts,bureau,business,capacity,cert,citrix,commander,computer,content,coordination,counter,crmportal,cyber,cybersecurity,data,database,deputy,desktopfield,developer,developerprogrammer,diem,edx,email,fellow,fisa,full,hardware,health,help,identity,incident,incidentproblemchange,information,initiatives,integration,intelligence,ios,it,lamp,lead,learning,linux,logging,logic,mainframe,management,manager,master,middleware,mobile,modeling,mwbe,net,network,of,peoplesoft,per,portal,positions,process,program,programmer,protection,psac,qa,record,recording,reliability,reporting,resiliency,saas,salesforce,scheduler,security,senior,sharepoint,software,solutions,specialist,stack,supplies,support,systems,task,technical,tester,threat,tivoli,upstate,vendor,vulnerability,web,workload,y,y_pred,true_pred,Job ID,is_it,Business Title,prob_pos,prob_neg
1713,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,385602,0,program executive,0.010928,0.989072
1366,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,379995,0,program coordinator,0.010928,0.989072
2446,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,395193,0,rentalfleet program processor,0.010928,0.989072


In [30]:
# уверенно предсказал класс 1, но ошибся
false_positive = false_pred[false_pred['y_pred'] == 1].sort_values(by = 'prob_pos', ascending=False).head(3)
print(false_positive['Business Title'].tolist())
false_positive

['senior .net developer,  bureau of application development and database administration', 'senior .net developer,  bureau of application development and database administration', '.net.developer, bureau of application development and database administration']


,analyst,application,applications,architectsenior,assistant,attorney,bim,bts,bureau,business,capacity,cert,citrix,commander,computer,content,coordination,counter,crmportal,cyber,cybersecurity,data,database,deputy,desktopfield,developer,developerprogrammer,diem,edx,email,fellow,fisa,full,hardware,health,help,identity,incident,incidentproblemchange,information,initiatives,integration,intelligence,ios,it,lamp,lead,learning,linux,logging,logic,mainframe,management,manager,master,middleware,mobile,modeling,mwbe,net,network,of,peoplesoft,per,portal,positions,process,program,programmer,protection,psac,qa,record,recording,reliability,reporting,resiliency,saas,salesforce,scheduler,security,senior,sharepoint,software,solutions,specialist,stack,supplies,support,systems,task,technical,tester,threat,tivoli,upstate,vendor,vulnerability,web,workload,y,y_pred,true_pred,Job ID,is_it,Business Title,prob_pos,prob_neg
1998,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,389880,0,"senior .net developer, bureau of application ...",0.996951,0.003049
1999,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,389880,0,"senior .net developer, bureau of application ...",0.996951,0.003049
1642,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,384504,0,".net.developer, bureau of application developm...",0.993070,0.006930


In [39]:
# уверенно предсказал класс 0, но ошибся
false_negative = false_pred[false_pred['y_pred'] == 0].sort_values(by = 'prob_neg', ascending=False).head(3)
print(false_negative['Business Title'].tolist())
false_negative

['capacity management engineer', 'director, program management office', 'director, program management office']


,analyst,application,applications,architectsenior,assistant,attorney,bim,bts,bureau,business,capacity,cert,citrix,commander,computer,content,coordination,counter,crmportal,cyber,cybersecurity,data,database,deputy,desktopfield,developer,developerprogrammer,diem,edx,email,fellow,fisa,full,hardware,health,help,identity,incident,incidentproblemchange,information,initiatives,integration,intelligence,ios,it,lamp,lead,learning,linux,logging,logic,mainframe,management,manager,master,middleware,mobile,modeling,mwbe,net,network,of,peoplesoft,per,portal,positions,process,program,programmer,protection,psac,qa,record,recording,reliability,reporting,resiliency,saas,salesforce,scheduler,security,senior,sharepoint,software,solutions,specialist,stack,supplies,support,systems,task,technical,tester,threat,tivoli,upstate,vendor,vulnerability,web,workload,y,y_pred,true_pred,Job ID,is_it,Business Title,prob_pos,prob_neg
127,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,378606,1,capacity management engineer,0.005946,0.994054
265,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,395178,1,"director, program management office",0.019018,0.980982
264,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,395178,1,"director, program management office",0.019018,0.980982


In [40]:
pd.concat([filtered_full, df_k_best[['y_pred', 'true_pred', 'prob_pos', 'prob_neg']]], axis=1, sort=False).head()

,Job ID,Agency,Posting Type,Business Title,Civil Service Title,Title Code No,Level,Job Category,Full-Time/Part-Time indicator,Salary Range From,Salary Range To,Salary Frequency,Work Location,Division/Work Unit,Job Description,Minimum Qual Requirements,Preferred Skills,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date,Mean Salary,is_it,y_pred,true_pred,prob_pos,prob_neg
0,177048,dept of info tech & telecomm,internal,application support reporting specialist,computer specialist software,13632,3,information technology & telecommunications,NaN,81290.0,95896.0,annual,2 metro tech,develop & integration services,"doitt provides for the sustained, efficient an...",1 a baccalaureate degree from an accredited co...,the preferred candidate should possess the fol...,NaN,for doitt employees only please go to emplo...,day - due to the necessary technical support d...,"brooklyn, ny",NaN,new york city residency is not required for th...,2014-12-02,NaN,12022014,07022019,88593.0,1,1,1,0.591830,0.408170
1,183112,dept of info tech & telecomm,internal,gis analyst,computer programmer analyst,13651,1,information technology & telecommunications,f,45174.0,62370.0,annual,2 metro tech,develop & integration services,"doitt provides for the sustained, efficient an...",1 a baccalaureate degree from an accredited co...,the successful candidate should possess the fo...,NaN,for doitt employees only please go to employe...,due to the necessary technical support duties ...,"brooklyn, ny",NaN,new york city residency is not required for th...,2015-02-12,NaN,02122015,07022019,53772.0,1,0,0,0.069033,0.930967
2,184328,dept of info tech & telecomm,internal,"director, infrastructure build coordinator",computer systems manager,10050,m1,information technology & telecommunications,NaN,49492.0,100000.0,annual,2 metro tech,public safety it services,"doitt provides for the sustained, efficient an...",1. a masters degree in computer science from a...,the successful candidate should possess the fo...,NaN,for doitt employees only please go to employe...,day - due to the necessary technical managemen...,"brooklyn, ny",NaN,new york city residency is not required for th...,2015-02-19,NaN,02192015,07022019,74746.0,1,0,0,0.035914,0.964086
3,229837,dept of info tech & telecomm,internal,senior quality oversight analyst,computer systems manager,10050,m2,information technology & telecommunications,f,58675.0,125000.0,annual,75 park place new york ny,general counsel,"doitt provides for the sustained, efficient an...",1. a masters degree in computer science from a...,the successful candidate possess the following...,NaN,for doitt employees only please go to employe...,day,manhattan,NaN,new york city residency is not required for th...,2016-02-17,NaN,02172016,07022019,91837.5,1,0,0,0.144724,0.855276
4,239204,admin for childrens svcs,external,child welfare analyst,city research scientist,21744,2,information technology & telecommunications po...,f,68239.0,85644.0,annual,"150 william street, new york n",srm eligrptsys spprt-p&p,the child welfare city research scientist will...,"1. for assignment level i only physical, biol...",critical to this position is strong technical ...,section 424-a of the new york social services ...,click on the apply now button.,NaN,NaN,NaN,new york city residency is generally required ...,2016-05-17,NaN,05172016,07022019,76941.5,1,0,0,0.069033,0.930967
